# Logistic Regression with a Neural Network mindset

Welcome to your first (required) programming assignment! You will build a logistic regression classifier to recognize  cats. This assignment will step you through how to do this with a Neural Network mindset, and so will also hone your intuitions about deep learning.

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Initializing parameters
    - Calculating the cost function and its gradient
    - Using an optimization algorithm (gradient descent) 
- Gather all three functions above into a main model function, in the right order.

## 1 - Packages ##

First, let's run the cell below to import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [h5py](http://www.h5py.org) is a common package to interact with a dataset that is stored on an H5 file.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) are used here to test your model with your own picture at the end.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import os
#from PIL import Image
from scipy import ndimage
import cv2
import random
%matplotlib inline

# Imagen como vector

La entrada para el modelo de regresión logística es una imagen. Una imagen es una matriz tridimensional que contiene valores de intensidad de píxel de rojo, verde y azul de los canales. En deep learning, lo que hacemos primero es que convirtamos esta imagen (3d-matriz) a una matriz de 1d (también llamado como un vector).
Por ejemplo, si una imagen es de [640, 480, 3], donde 640 es anchura, 480 es altura y 3 es el número de canales, enttonces la versión vector sería [1, 640x480x3]

![title](image-to-vector.jpg)

In [2]:
# Reduje el tamaño del train ya que mi laptop se cuelga
train_files = os.listdir("train")
lista = []
for i in range(1000):
    lista.append(train_files[i])
    lista.append(train_files[-i])

In [3]:
lista

['cat.0.jpg',
 'cat.0.jpg',
 'cat.1.jpg',
 'dog.9999.jpg',
 'cat.10.jpg',
 'dog.9998.jpg',
 'cat.100.jpg',
 'dog.9997.jpg',
 'cat.1000.jpg',
 'dog.9996.jpg',
 'cat.10000.jpg',
 'dog.9995.jpg',
 'cat.10001.jpg',
 'dog.9994.jpg',
 'cat.10002.jpg',
 'dog.9993.jpg',
 'cat.10003.jpg',
 'dog.9992.jpg',
 'cat.10004.jpg',
 'dog.9991.jpg',
 'cat.10005.jpg',
 'dog.9990.jpg',
 'cat.10006.jpg',
 'dog.999.jpg',
 'cat.10007.jpg',
 'dog.9989.jpg',
 'cat.10008.jpg',
 'dog.9988.jpg',
 'cat.10009.jpg',
 'dog.9987.jpg',
 'cat.1001.jpg',
 'dog.9986.jpg',
 'cat.10010.jpg',
 'dog.9985.jpg',
 'cat.10011.jpg',
 'dog.9984.jpg',
 'cat.10012.jpg',
 'dog.9983.jpg',
 'cat.10013.jpg',
 'dog.9982.jpg',
 'cat.10014.jpg',
 'dog.9981.jpg',
 'cat.10015.jpg',
 'dog.9980.jpg',
 'cat.10016.jpg',
 'dog.998.jpg',
 'cat.10017.jpg',
 'dog.9979.jpg',
 'cat.10018.jpg',
 'dog.9978.jpg',
 'cat.10019.jpg',
 'dog.9977.jpg',
 'cat.1002.jpg',
 'dog.9976.jpg',
 'cat.10020.jpg',
 'dog.9975.jpg',
 'cat.10021.jpg',
 'dog.9974.jpg',
 'cat.

In [4]:
train_data = []
for i, im in enumerate(lista):
    filename = "train/" + im
    image = np.asarray(cv2.imread(filename))
    train_data.append((image, 1 if im.split(".")[0] == "cat" else 0))

In [5]:
print("Las dimensiones de 10 imagenes random")
for _ in range(10):
    print(random.choice(train_data)[0].shape)

Las dimensiones de 10 imagenes random
(263, 349, 3)
(258, 214, 3)
(421, 499, 3)
(383, 499, 3)
(499, 353, 3)
(374, 500, 3)
(357, 499, 3)
(417, 423, 3)
(309, 333, 3)
(374, 500, 3)


Ahora vamos a hacer que cada imagen tenga la misma dimensión.
Para esto averiguemos cuantas imágenes tienen >= 64 píxeles

In [6]:
c = 0
for x, y in train_data:
    c += x.shape[0] >= 64 and x.shape[1] >= 64
c

1994

Teniendo en cuanta que el tamaño de nuestro train es de 2000, 1994 representa más del 99%.

Este enfoque es simplemente un enfoque ingenuo que adopté para hacer todas las imágenes del mismo tamaño y sentí que la reducción de escala no degradaría la calidad de las imágenes tanto como lo haría la ampliación de escala, ya que la ampliación de una imagen muy pequeña a una grande, principalmente a efectos pixelados y haría más difícil el aprendizaje para el modelo. Además, esta dimensión64 por 64 por 3 no es un número mágico, simplemente es algo con lo que fui.

Continuemos y veamos el código que cambiaría el tamaño de todas estas imágenes y también dividiría los datos en conjuntos de test y train. Dividimos los datos proporcionados mediante una división de 80/20, es decir, el 80% de los datos se utilizaría para capacitar a nuestros datos y el 20% restante se usaría para probar el modelo para ver el rendimiento final de los datos.

In [7]:
train_size = int(len(lista)*0.8)
train_data_x = np.zeros((train_size, 64, 64, 3))
train_data_y = np.zeros((1, train_size))
valid_data_x = np.zeros((len(lista) - train_size, 64, 64, 3))
valid_data_y = np.zeros((1, len(lista) - train_size))

In [8]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
for i, (x, y) in enumerate(train_data):
    resize_image = np.resize(x, (64, 64, 3))
    if i<train_size:
        train_data_x[i] = resize_image
        train_data_y[:, i] = y
    else:
        valid_data_x[i - train_size] = resize_image
        valid_data_y[:, i - train_size] = y

In [9]:
train_data_x.shape, train_data_y.shape, valid_data_x.shape, valid_data_y.shape

((1600, 64, 64, 3), (1, 1600), (400, 64, 64, 3), (1, 400))

Ahora que hemos procesado nuestros datos y los tenemos en el formato que necesitamos, finalmente podemos guardarlos en dos archivos separados: train.npz y valid.npz.

In [10]:
np.savez("train", X = train_data_x, Y = train_data_y)
np.savez("valid", X = valid_data_x, Y = valid_data_y)

In [11]:
def load_kaggle_dataset():
    train = np.load('train.npz')
    valid = np.load('valid.npz')
    train_x_original, train_y = train['X'], train['Y']
    valid_x_original, valid_y = valid['X'], valid['Y']
    return train_x_original, train_y, valid_x_original, valid_y

In [12]:
def load_kaggle_dataset():
    train = np.load('train.npz')
    valid = np.load('valid.npz')
    train_x_original, train_y = train['X'], train['Y']
    valid_x_original, valid_y = valid['X'], valid['Y']
    return train_x_original, train_y, valid_x_original, valid_y

In [13]:
train_x_original, train_y, valid_x_original, valid_y = load_kaggle_dataset()

In [14]:
def image2vec(image_rgb_matrix):
    return image_rgb_matrix.reshape(image_rgb_matrix.shape[0], -1).T

In [16]:
train_x = image2vec(train_x_original)
valid_x = image2vec(valid_x_original)
print(train_x)
print(train_x.shape)
print("control de cordura después de remodelar:" + str(train_x[0:5,0]))

[[ 87.  87.  40. ... 158.  67.  10.]
 [164. 164.  44. ... 133.  36.   9.]
 [203. 203.  39. ... 131.  33.  11.]
 ...
 [132. 132. 125. ... 110. 107.  66.]
 [214. 214. 115. ... 125. 126. 106.]
 [249. 249. 145. ... 144. 171. 155.]]
(12288, 1600)
control de cordura después de remodelar:[ 87. 164. 203.  87. 164.]


In [17]:
"""Normalizar los datos"""
train_x /= 255
valid_x /= 255

In [49]:
train_x.shape

(12288, 1600)

In [18]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [19]:
def initialize_parameters(dim):
    w = np.zeros((dim, 1))
    b = 0
    
    assert (w.shape == (dim, 1))
    assert (isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [20]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    cost = (-1 / m)*np.sum(Y * np.log(A) + (1 - Y)*np.log(1 - A))
    
    #Backpropagation
    
    dw = (1 / m)*np.dot(X, (A - Y).T) # dw es la derivada de la pérdida respecto al peso w
    
    db = (1 / m)*np.sum(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [21]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601]
 [2.39507239]]
db = 0.001455578136784208
cost = 5.801545319394553


In [31]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costo = []
    
    for i in range(num_iterations):
    
        grads, cost = propagate(w, b, X, Y)
    
        dw = grads['dw']
        db = grads['db']
    
        w = w - learning_rate*dw
        b = b - learning_rate*db
    
        if i % 100 == 0:
            costo.append(cost)
    
        if print_cost and i % 100 == 0:
            print("Costo después de %i iteraciones: %f" %(i, cost))
        
    parametros = {"w": w, "b": b}
    gradientes = {"dw": dw, "db": db}
    
    return parametros, gradientes, costo

In [32]:
parametros, gradientes, costo = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = True)

print ("w = " + str(parametros["w"]))
print ("b = " + str(parametros["b"]))
print ("dw = " + str(gradientes["dw"]))
print ("db = " + str(gradientes["db"]))

Costo después de 0 iteraciones: 5.801545
w = [[0.19033591]
 [0.12259159]]
b = 1.9253598300845747
dw = [[0.67752042]
 [1.41625495]]
db = 0.21919450454067657


In [47]:
def prediccion(w, b, X):
    
    """
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    X -> shape = (num_px*num_px*3, m)
    m -> número de ejemplos
    """
    
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape((X.shape[0], 1))
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        
        Y_predict = np.round(A)
        
    assert(Y_predict.shape == (1, m))
    
    return Y_predict

In [48]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(prediccion(w, b, X)))

predictions = [[1. 1. 0.]]


In [56]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    
    w, b = initialize_parameters(X_train.shape[0])
    
    parametros, gradiente, costo = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parametros['w']
    b = parametros['b']
    
    Y_prediction_train = prediccion(w, b, X_train)
    Y_prediction_test = prediccion(w, b, X_test)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costo,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}

In [57]:
d = model(train_x, train_y, valid_x, valid_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Costo después de 0 iteraciones: 0.693147
Costo después de 100 iteraciones: 1.502369
Costo después de 200 iteraciones: 1.475272
Costo después de 300 iteraciones: 1.449972
Costo después de 400 iteraciones: 1.426826
Costo después de 500 iteraciones: 1.405986
Costo después de 600 iteraciones: 1.387152
Costo después de 700 iteraciones: 1.369923
Costo después de 800 iteraciones: 1.353943
Costo después de 900 iteraciones: 1.338934
Costo después de 1000 iteraciones: 1.324696
Costo después de 1100 iteraciones: 1.311083
Costo después de 1200 iteraciones: 1.297991
Costo después de 1300 iteraciones: 1.285346
Costo después de 1400 iteraciones: 1.273097
Costo después de 1500 iteraciones: 1.261203
Costo después de 1600 iteraciones: 1.249636
Costo después de 1700 iteraciones: 1.238374
Costo después de 1800 iteraciones: 1.227401
Costo después de 1900 iteraciones: 1.216704
train accuracy: 53.375 %
test accuracy: 48.25000000000001 %
